# Changing Music Preferences
### Do Spotify users listen to different music during the COVID-19 pandemic? 
### Does this relate to the amount of confirmed cases?

To Do: 

1. Scraping Spotify Top 200 data
2. Get song metadata through Spotify API
3. Data Visualization & Modelling
4. .....
5. Create interactive dashboard in Shiny


## 1.
## Scraping the Top 200 data for 2019 (baseline) and 2020 (pandemic).

https://spotifycharts.com contains the official Spotify Charts. 

Luckily, each day/week is available as a .csv file!
<br> <br> <br>
**Goal: get all the weekly .csv files from 2019 and 2020 and combine them to one.**

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import io

In [3]:
url = 'https://spotifycharts.com/regional/nl/weekly/latest'

r = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [4]:
dates = [item["data-value"] for item in soup.find_all(attrs={"data-value": True}) if item["data-value"].startswith('2')]
dates[:5]

['2020-10-30--2020-11-06',
 '2020-10-23--2020-10-30',
 '2020-10-16--2020-10-23',
 '2020-10-09--2020-10-16',
 '2020-10-02--2020-10-09']

In [5]:
urls = ["https://spotifycharts.com/regional/nl/weekly/" + date + "/download" for date in dates]
urls[:5]

['https://spotifycharts.com/regional/nl/weekly/2020-10-30--2020-11-06/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-10-23--2020-10-30/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-10-16--2020-10-23/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-10-09--2020-10-16/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-10-02--2020-10-09/download']

### Warning: running the block below will result in scraping 200 .csv's!

In [ ]:
data = []

for date in tqdm(dates): 
    url = "https://spotifycharts.com/regional/nl/weekly/" + date + "/download"
    response = requests.get(url)
    file_object = io.StringIO(response.content.decode('utf-8'))
    df = pd.read_csv(file_object, header=1)
    df["Date"] = date
    data.append(df)

df = pd.concat(data)
df.reset_index(drop=True, inplace=True)
df.to_csv("..\\data\\raw\\top200_2017_2020.csv", index=False)
data = []

In [6]:
df = pd.read_csv("..\\data\\raw\\top200_2017_2020.csv")
df

,Position,Track Name,Artist,Streams,URL,Date
0,1,Scooter,Yssi SB,2375687,https://open.spotify.com/track/4eWeOwo0TB0vkdr...,2020-10-23--2020-10-30
1,2,Mood (feat. iann dior),24kGoldn,1614485,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-23--2020-10-30
2,3,"Lemonade (feat. Gunna, Don Toliver & NAV)",Internet Money,1429665,https://open.spotify.com/track/7hxHWCCAIIxFLCz...,2020-10-23--2020-10-30
3,4,Lonely (with benny blanco),Justin Bieber,1330793,https://open.spotify.com/track/4y4spB9m0Q6026K...,2020-10-23--2020-10-30
4,5,Head & Heart (feat. MNEK),Joel Corry,1271413,https://open.spotify.com/track/6cx06DFPPHchuUA...,2020-10-23--2020-10-30
...,...,...,...,...,...,...
40195,196,Sex,Cheat Codes,114030,https://open.spotify.com/track/5DA77EqppDmCTWG...,2016-12-23--2016-12-30
40196,197,Ain't My Fault,Zara Larsson,113974,https://open.spotify.com/track/0ADG9OgdVTL7fgR...,2016-12-23--2016-12-30
40197,198,Please Come Home for Christmas,Luther Vandross,113779,https://open.spotify.com/track/2mOtx6P21hecOcP...,2016-12-23--2016-12-30
40198,199,Jodge Me Niet - Titelsong Van De Film “SOOF 2”,Jayh,113763,https://open.spotify.com/track/2VxAfqI3vIOaPSl...,2016-12-23--2016-12-30


In [7]:
print(df.isna().sum())
df.dropna(inplace=True) # removing 8 rows out of 40k is justified

df[['Start Week', 'End Week']] = df['Date'].str.split('--', 1, expand=True) # add columns for start and end dates
df['ID'] = df['URL'].str.rsplit('https://open.spotify.com/track/', 1, expand=True)[1] # add ID column for later merge

df.to_csv("..\\data\\processed\\top200_2017_2020.csv", index=False)

Position      0
Track Name    7
Artist        7
Streams       0
URL           2
Date          0
dtype: int64


In [8]:
df = pd.read_csv("..\\data\\processed\\top200_2017_2020.csv")
df.head()

,Position,Track Name,Artist,Streams,URL,Date,Start Week,End Week,ID
0,1,Scooter,Yssi SB,2375687,https://open.spotify.com/track/4eWeOwo0TB0vkdr...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,4eWeOwo0TB0vkdrUhy4zbM
1,2,Mood (feat. iann dior),24kGoldn,1614485,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,3tjFYV6RSFtuktYl3ZtYcq
2,3,"Lemonade (feat. Gunna, Don Toliver & NAV)",Internet Money,1429665,https://open.spotify.com/track/7hxHWCCAIIxFLCz...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,7hxHWCCAIIxFLCzvDgnQHX
3,4,Lonely (with benny blanco),Justin Bieber,1330793,https://open.spotify.com/track/4y4spB9m0Q6026K...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,4y4spB9m0Q6026KfkAvy9Q
4,5,Head & Heart (feat. MNEK),Joel Corry,1271413,https://open.spotify.com/track/6cx06DFPPHchuUA...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,6cx06DFPPHchuUAcTxznu9


In [17]:
len(df['ID'].unique())

5044

## 2.
## Retrieve and Combine music features through Spotify API


In [11]:
# Fill this in using your Spotify API Credentials from https://developer.spotify.com/dashboard/applications
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="XXX" 
secret = "XXX"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
# Where I save my Spotify API Credentials, outside of this public repository :)
import spotipy
%run credentials.py

In [20]:
# Inspired by https://github.com/tgel0/spotify-data/blob/master/notebooks/SpotifyDataRetrieval.ipynb under MIT Licence.

df_ids = df["ID"].drop_duplicates()

rows = []
batchsize = 100
None_counter = 0

for i in tqdm(range(0,len(df_ids),batchsize)):
    batch = df_ids[i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)

100%|██████████| 51/51 [00:07<00:00,  7.01it/s]


In [22]:
features = pd.DataFrame.from_dict(rows, orient="columns")
features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.781,0.412,0,-10.860,1,0.1410,0.262,0.000000,0.4280,0.8790,159.981,audio_features,4eWeOwo0TB0vkdrUhy4zbM,spotify:track:4eWeOwo0TB0vkdrUhy4zbM,https://api.spotify.com/v1/tracks/4eWeOwo0TB0v...,https://api.spotify.com/v1/audio-analysis/4eWe...,278000,4
1,0.700,0.722,7,-3.558,0,0.0369,0.221,0.000000,0.2720,0.7560,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4
2,0.800,0.658,1,-6.142,0,0.0790,0.250,0.000000,0.1110,0.4620,140.042,audio_features,7hxHWCCAIIxFLCzvDgnQHX,spotify:track:7hxHWCCAIIxFLCzvDgnQHX,https://api.spotify.com/v1/tracks/7hxHWCCAIIxF...,https://api.spotify.com/v1/audio-analysis/7hxH...,195429,4
3,0.631,0.239,11,-7.071,0,0.0398,0.864,0.000000,0.1160,0.0927,79.859,audio_features,4y4spB9m0Q6026KfkAvy9Q,spotify:track:4y4spB9m0Q6026KfkAvy9Q,https://api.spotify.com/v1/tracks/4y4spB9m0Q60...,https://api.spotify.com/v1/audio-analysis/4y4s...,149297,4
4,0.734,0.874,8,-3.158,1,0.0662,0.168,0.000011,0.0489,0.9050,122.953,audio_features,6cx06DFPPHchuUAcTxznu9,spotify:track:6cx06DFPPHchuUAcTxznu9,https://api.spotify.com/v1/tracks/6cx06DFPPHch...,https://api.spotify.com/v1/audio-analysis/6cx0...,166028,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5039,0.622,0.688,4,-3.504,0,0.0845,0.520,0.000000,0.3750,0.7660,140.561,audio_features,18g50kFt5uwVeCzQbrCvAO,spotify:track:18g50kFt5uwVeCzQbrCvAO,https://api.spotify.com/v1/tracks/18g50kFt5uwV...,https://api.spotify.com/v1/audio-analysis/18g5...,161973,4
5040,0.664,0.518,11,-9.947,1,0.0335,0.266,0.000031,0.0762,0.3300,80.969,audio_features,1DGYmHLYkwyCmvDEL3ctti,spotify:track:1DGYmHLYkwyCmvDEL3ctti,https://api.spotify.com/v1/tracks/1DGYmHLYkwyC...,https://api.spotify.com/v1/audio-analysis/1DGY...,242893,4
5041,0.295,0.496,7,-9.478,1,0.0327,0.207,0.000396,0.8580,0.2240,132.062,audio_features,4QVdnpSO5JGMFOEl6ys4RQ,spotify:track:4QVdnpSO5JGMFOEl6ys4RQ,https://api.spotify.com/v1/tracks/4QVdnpSO5JGM...,https://api.spotify.com/v1/audio-analysis/4QVd...,268521,4
5042,0.325,0.759,8,-7.146,1,0.0530,0.395,0.000362,0.0773,0.3600,126.627,audio_features,46pF1zFimM582ss1PrMy68,spotify:track:46pF1zFimM582ss1PrMy68,https://api.spotify.com/v1/tracks/46pF1zFimM58...,https://api.spotify.com/v1/audio-analysis/46pF...,166453,4


In [23]:
features.drop(['analysis_url','track_href','type','uri'], axis=1,inplace=True)

In [24]:
features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,5044.000000,5044.000000,5044.000000,5044.000000,5044.000000,5044.000000,5044.000000,5044.000000,5044.000000,5044.000000,5044.000000,5.044000e+03,5044.000000
mean,0.705734,0.641500,5.329897,-6.702512,0.518438,0.166293,0.228704,0.009184,0.167508,0.521988,119.226200,1.914033e+05,3.991673
std,0.137359,0.152487,3.629398,2.328562,0.499709,0.130550,0.225705,0.063489,0.124543,0.211665,28.813732,4.399746e+04,0.323466
min,0.153000,0.031600,0.000000,-23.023000,0.000000,0.023200,0.000055,0.000000,0.018300,0.036000,38.796000,3.013300e+04,1.000000
25%,0.617000,0.552000,2.000000,-7.787250,0.000000,0.054600,0.053675,0.000000,0.093600,0.360000,97.917500,1.645958e+05,4.000000
50%,0.725000,0.650000,6.000000,-6.375500,1.000000,0.120000,0.149000,0.000000,0.120000,0.531000,114.949500,1.880200e+05,4.000000
75%,0.808000,0.753000,8.000000,-5.212750,1.000000,0.259000,0.336000,0.000039,0.194000,0.683000,138.055750,2.133150e+05,4.000000
max,0.974000,0.990000,11.000000,-0.793000,1.000000,0.966000,0.993000,0.918000,0.973000,0.989000,216.821000,1.336000e+06,5.000000


In [25]:
print(features.isna().sum())
print(features.info())

danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
id                  0
duration_ms         0
time_signature      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5044 entries, 0 to 5043
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      5044 non-null   float64
 1   energy            5044 non-null   float64
 2   key               5044 non-null   int64  
 3   loudness          5044 non-null   float64
 4   mode              5044 non-null   int64  
 5   speechiness       5044 non-null   float64
 6   acousticness      5044 non-null   float64
 7   instrumentalness  5044 non-null   float64
 8   liveness          5044 non-null   float64
 9   valence           5044 non-null   float64
 10  tem

In [26]:
df_merge = pd.merge(df, features, left_on="ID", right_on="id", how="inner")
df_merge

,Position,Track Name,Artist,Streams,URL,Date,Start Week,End Week,ID,danceability,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,1,Scooter,Yssi SB,2375687,https://open.spotify.com/track/4eWeOwo0TB0vkdr...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,4eWeOwo0TB0vkdrUhy4zbM,0.781,...,1,0.1410,0.262,0.000000,0.4280,0.879,159.981,4eWeOwo0TB0vkdrUhy4zbM,278000,4
1,2,Mood (feat. iann dior),24kGoldn,1614485,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
2,1,Mood (feat. iann dior),24kGoldn,1637451,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-16--2020-10-23,2020-10-16,2020-10-23,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
3,1,Mood (feat. iann dior),24kGoldn,1693876,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-09--2020-10-16,2020-10-09,2020-10-16,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
4,1,Mood (feat. iann dior),24kGoldn,1787537,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-02--2020-10-09,2020-10-02,2020-10-09,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40188,174,Man With The Bag,Jessie J,123048,https://open.spotify.com/track/18g50kFt5uwVeCz...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,18g50kFt5uwVeCzQbrCvAO,0.622,...,0,0.0845,0.520,0.000000,0.3750,0.766,140.561,18g50kFt5uwVeCzQbrCvAO,161973,4
40189,176,Christmas Day,Dido,122741,https://open.spotify.com/track/1DGYmHLYkwyCmvD...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,1DGYmHLYkwyCmvDEL3ctti,0.664,...,1,0.0335,0.266,0.000031,0.0762,0.330,80.969,1DGYmHLYkwyCmvDEL3ctti,242893,4
40190,182,Christmas Lights - Live from Spotify London,Coldplay,119168,https://open.spotify.com/track/4QVdnpSO5JGMFOE...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,4QVdnpSO5JGMFOEl6ys4RQ,0.295,...,1,0.0327,0.207,0.000396,0.8580,0.224,132.062,4QVdnpSO5JGMFOEl6ys4RQ,268521,4
40191,195,Christmas (Baby Please Come Home),Darlene Love,114594,https://open.spotify.com/track/46pF1zFimM582ss...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,46pF1zFimM582ss1PrMy68,0.325,...,1,0.0530,0.395,0.000362,0.0773,0.360,126.627,46pF1zFimM582ss1PrMy68,166453,4


In [27]:
df_merge.to_csv("..\\data\\processed\\top200_2017_2020_w_features.csv", index=False)

## 3.
## Data Visualization and Modelling
## To Do: Normalize features!

In [84]:
# Load the data and transform date-time columns
df = pd.read_csv("..\\data\\processed\\top200_2017_2020_w_features.csv")
df[['Start Week', 'End Week']] = df[['Start Week', 'End Week']].apply(pd.to_datetime, format="%Y-%m-%d")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40193 entries, 0 to 40192
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Position          40193 non-null  int64         
 1   Track Name        40193 non-null  object        
 2   Artist            40193 non-null  object        
 3   Streams           40193 non-null  int64         
 4   URL               40193 non-null  object        
 5   Date              40193 non-null  object        
 6   Start Week        40193 non-null  datetime64[ns]
 7   End Week          40193 non-null  datetime64[ns]
 8   ID                40193 non-null  object        
 9   danceability      40193 non-null  float64       
 10  energy            40193 non-null  float64       
 11  key               40193 non-null  int64         
 12  loudness          40193 non-null  float64       
 13  mode              40193 non-null  int64         
 14  speechiness       4019

In [95]:
avg_month = df[['Start Week', 'danceability', 'energy', 'key', 
 'loudness', 'mode', 'speechiness', 'acousticness', 
 'instrumentalness', 'liveness', 'valence', 'tempo', 
 'duration_ms']].groupby(pd.Grouper(key="Start Week", freq='M')).mean()

avg_month["Y"] = avg_month.index.year
avg_month["M"] = avg_month.index.month
avg_month["D"] = avg_month.index.day
avg_month["YMD"] = avg_month.index.date

avg_month.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,Y,M,D,YMD
Start Week,,,,,,,,,,,,,,,,
2016-12-31,0.652360,0.669225,5.47000,-6.434782,0.57250,0.115267,0.239128,0.003734,0.173714,0.528089,118.008208,209940.5325,2016,12,31,2016-12-31
2017-01-31,0.677675,0.679245,5.35250,-6.168275,0.52500,0.126757,0.209052,0.006865,0.161820,0.509564,117.173697,211221.7875,2017,1,31,2017-01-31
2017-02-28,0.680190,0.684178,5.28750,-6.046697,0.53250,0.128803,0.199814,0.007220,0.161475,0.516377,117.279955,211037.7325,2017,2,28,2017-02-28
2017-03-31,0.688275,0.684836,5.42200,-5.885204,0.56600,0.141018,0.206340,0.008309,0.161549,0.528210,116.246525,209765.2090,2017,3,31,2017-03-31
2017-04-30,0.690864,0.690809,5.31375,-5.774167,0.54625,0.147161,0.204996,0.006361,0.154938,0.542373,116.230464,209821.9625,2017,4,30,2017-04-30


In [90]:
import plotly.express as px

fig = px.line(avg_month, x="M", y="energy", color='Y')
fig.show()

In [94]:
fig = px.line(avg_month, x="YMD", y="danceability", color='Y')
fig.show()